# BSSN-puncture camppanelly et al. 2008

Substitute $\chi := \text{exp}(-4\phi)$ to the classic BSSN method


$$
\begin{align}
{\color{red}\partial_t \tilde\gamma_{ij} =}& -2 \alpha \tilde A_{ij} + \mathcal{L}_{\beta}\tilde \gamma_{ij} 
\\
\partial_t \chi =& \frac{2}{3}\chi( \alpha K - \partial_a \beta^a) + \beta^i\partial_i \chi 
\\
\partial_0 \tilde A_{ij} =&\mathcal{L}_{\beta}A_{ij} + \chi \bigg[ -D_i D_j \alpha + \alpha R_{ij}\bigg]^{TF}+ \alpha \bigg[ K\tilde A_{ij } - 2 \tilde A_{il} \tilde A^l_j \bigg] 
\\
\partial_0 K =& -D^i D_i \alpha + \alpha \bigg[ \tilde A_{ij} \tilde A^{ij} + \frac{1}{3} K^2 \bigg] 
\\
\partial_t \tilde\Gamma^i =& \tilde \gamma^{jk} \bigg[ \frac{2}{3} \partial_j \partial_k \beta^i - 2 \partial_j \beta^l \Gamma^i_{kl} + \beta^l \partial_l \Gamma^i_{jk} \bigg] + \frac{1}{3} \tilde \gamma^{ik} \partial_j \partial_k \beta^i 
+ \frac{1}{3}  \tilde \gamma^{ij} \partial_j \partial_k \beta^k 
+ \beta^j \partial_j \tilde \Gamma^i 
- \tilde \Gamma^i \partial_j \beta^j 
\end{align}
$$
with 
$\partial_0 := \partial_t - \mathcal{L}_\beta$,


In [85]:
from ngsolve import *
from zenith import * 
from ngsolve import *
from netgen.csg import *
import scipy.sparse as sp
import matplotlib.pylab as plt



import netgen.webgui  
from ngsolve.webgui import Draw
#import netgen.gui
#%gui tk

In [86]:
# standard mesh
with TaskManager():
    # create a mesh
    h = 0.3
    r = 0.8
    H = 1
    R = 3
    order = 2
    kwargs = {"bonus_intorder": 10, "inverse": "pardiso", "order": order, "mesh_order": 1}

    # BH1
    pos1 =  (1,0,0)
    lin1 = (0,1,0)
    ang1 = (0,0,1)
    mass1 =1
    BH1 = BlackHole(mass1, pos1, lin1, ang1 )

    # BH2
    pos2 =  (-1,0,0)
    lin2 = (0,-1,0)
    ang2 = (0,0,1)
    mass2 =1
    BH2 = BlackHole(mass2, pos2, lin2, ang2 )

    BHs = [BH1, BH2]

    mesh = MeshBlackHoles(BHs, h=h, R=R, H = H, r= r,  curve_order = 2)

#Draw(mesh, clipping= {"z":-1})

In [87]:
scl_h1 = H1(mesh, order=order)#, dirichlet="outer")
vec_h1 = VectorValued(scl_h1, 3)
mat_h1 = VectorValued(scl_h1, 9)

s, ds = scl_h1.TnT()
v, dv = vec_h1.TnT()
m, dm = mat_h1.TnT()

gf_g     = GridFunction(mat_h1)
gf_chi   = GridFunction(scl_h1)
gf_A     = GridFunction(mat_h1)
gf_K     = GridFunction(scl_h1)
gf_G     = GridFunction(vec_h1)
gf_alpha = GridFunction(scl_h1)
gf_beta  = GridFunction(vec_h1)
gf_B     = GridFunction(vec_h1)
gf_phi   = GridFunction(scl_h1)

In [88]:
fes = gf_g.space * gf_chi.space * gf_A.space * gf_K.space * gf_G.space * gf_alpha.space * gf_beta.space * gf_B.space * gf_phi.space

In [89]:
# set a positive inital condition all the variables
with TaskManager():
    def Peak(pnt = (0,0,0) ,**kwargs) : 
        return exp(-kwargs.get("sigma", 1)* ( (x-pnt[0])**2 + (y-pnt[1])**2 + (z-pnt[2])**2 ) )
    pnt1 = (-1, 0 ,0)
    pnt2 = (1, 0 ,0)
    pnt0 = (0, 0 ,0)
    
    peak1 = Peak(pnt = pnt1)
    peak2 = Peak(pnt = pnt2)
    peak0 = Peak(pnt = pnt0)
    
    bio = 10
    
    gf_g.Set( (peak0,0,0, 0,peak0, 0 , 0,0 , peak0 )  , bonus_intorder = bio)
    gf_chi.Set( (peak0+peak1 + peak2)  , bonus_intorder=bio)
    gf_A.Set( (0,0,peak1 +peak2, peak1 +peak2,0,0, 0,0,0)  , bonus_intorder=bio)
    gf_K.Set( (1/(1+peak1 + peak2) )  , bonus_intorder=bio)
    gf_G.Set( (peak0,peak1,peak2)  , bonus_intorder=bio)
    
    gf_alpha.Set( ( 1/(1+peak1 +peak2))  , bonus_intorder=bio)
    gf_beta.Set( (peak2,peak0,peak1)  , bonus_intorder=bio)
    gf_B.Set( (peak1,peak2,peak0)  , bonus_intorder=bio)
    gf_phi.Set((-log(peak0+peak1 + peak2)/4))
    
    
    

In [90]:
#gf_out = GridFunction(fes)
gf = GridFunction(fes)

# work only with components 0, 5
gf.components[0].vec.data = gf_g.vec
gf.components[1].vec.data = gf_chi.vec
gf.components[2].vec.data = gf_A.vec
gf.components[3].vec.data = gf_K.vec
gf.components[4].vec.data = gf_G.vec
gf.components[5].vec.data = gf_alpha.vec
gf.components[6].vec.data = gf_beta.vec
gf.components[7].vec.data = gf_B.vec
gf.components[8].vec.data = gf_phi.vec

#Draw(Norm(gf.components[0]), mesh, "g",  clipping= {"z":-1})
#Draw(Norm(gf.components[1]), mesh, "chi",  clipping= {"z":-1})
#Draw(Norm(gf.components[2]), mesh, "A",  clipping= {"z":-1})
#Draw(Norm(gf.components[3]), mesh, "K",  clipping= {"z":-1})
#Draw(Norm(f.components[4]), mesh, "G",  clipping= {"z":-1})
#Draw(Norm(gf.components[5]), mesh, "alpha",  clipping= {"z":-1})
#Draw(Norm(gf.components[6]), mesh, "beta",  clipping= {"z":-1})
#Draw(Norm(gf.components[7]), mesh, "B",  clipping= {"z":-1})
# Draw(Norm(gf.components[8]), mesh, "phi")

In [91]:
# use embedding to create inverse of whole matrix
inverse = "sparsecholesky"

with TaskManager():
    ## g
    mass_g = BilinearForm(mat_h1, symmetric=True)
    mass_g += InnerProduct(m, dm)*dx
    mass_g.Assemble()
    mass_g_inv = mass_g.mat.Inverse(inverse=inverse)
    res_g = fes.restrictions[0]
    inv_g = res_g.T@mass_g_inv@res_g

    ## chi
    mass_chi = BilinearForm(scl_h1, symmetric=True)
    mass_chi += InnerProduct(s, ds)*dx
    mass_chi.Assemble()
    mass_chi_inv = mass_chi.mat.Inverse(inverse=inverse)
    res_chi = fes.restrictions[1]
    inv_chi = res_chi.T @ mass_chi_inv @ res_chi

    ## A
    mass_A = BilinearForm(mat_h1, symmetric=True)
    mass_A += InnerProduct(m, dm)*dx
    mass_A.Assemble()
    mass_A_inv = mass_A.mat.Inverse(inverse=inverse)
    res_A = fes.restrictions[2]
    inv_A = res_A.T @ mass_A_inv @ res_A

    ## K
    mass_K = BilinearForm(scl_h1, symmetric=True)
    mass_K += InnerProduct(s, ds)*dx
    mass_K.Assemble()
    mass_K_inv = mass_K.mat.Inverse(inverse=inverse)
    res_K = fes.restrictions[3]
    inv_K = res_K.T@mass_K_inv@res_K

    ## G
    mass_G = BilinearForm(vec_h1, symmetric=True)
    mass_G += InnerProduct(v, dv)*dx
    mass_G.Assemble()
    mass_G_inv = mass_G.mat.Inverse(inverse=inverse)
    res_G = fes.restrictions[4]
    inv_G = res_G.T@mass_G_inv@res_G

    ## alpha
    mass_alpha = BilinearForm(scl_h1, symmetric=True)
    mass_alpha += InnerProduct(s, ds)*dx
    mass_alpha.Assemble()
    mass_alpha_inv = mass_alpha.mat.Inverse(inverse=inverse)
    res_alpha = fes.restrictions[5]
    inv_alpha = res_alpha.T@mass_alpha_inv@res_alpha

    ## beta
    mass_beta = BilinearForm(vec_h1, symmetric=True)
    mass_beta += InnerProduct(v, dv)*dx
    mass_beta.Assemble()
    mass_beta_inv = mass_beta.mat.Inverse(inverse=inverse)
    res_beta = fes.restrictions[6]
    inv_beta = res_beta.T@mass_beta_inv@res_beta

    ## Bs
    mass_B = BilinearForm(vec_h1, symmetric=True)
    mass_B += InnerProduct(v, dv)*dx
    mass_B.Assemble()
    mass_B_inv = mass_B.mat.Inverse(inverse=inverse)
    res_B = fes.restrictions[7]
    inv_B = res_B.T@mass_B_inv@res_B
    
    ## phi
    mass_phi = BilinearForm(scl_h1, symmetric=True)
    mass_phi += InnerProduct(s, ds)*dx
    mass_phi.Assemble()
    mass_phi_inv = mass_phi.mat.Inverse(inverse=inverse)
    res_phi = fes.restrictions[8]
    inv_phi = res_phi.T@mass_phi_inv@res_phi





In [92]:
def Lie(gf_M, gf_b):
    if gf_M.dim == 9:
        r =  grad(gf_M).Reshape((3,3,3)) * gf_b + grad(gf_b) * gf_M.Reshape((3,3)) + gf_M.Reshape((3,3)) *  grad(gf_b)  - 2/3*  div(gf_b) * gf_M.Reshape((3,3)) 
        return r.Reshape((9,))
    if gf_M.dim == 3:
        r =  grad(gf_M).Reshape((3,3)) * gf_b
        return r.Reshape((3,))
    

In [93]:


# define the christoffel symbols of the first kind
def CS1(gf_g):
    print( Grad(gf_g) )
    gradg = Grad(gf_g).Reshape((3,3,3))
    # Christoffel symbols of the first kind
    # cs[i,j,k] = 1/2 * (gradg[i,k,j] + gradg[k,j,i]  -gradg[i,j,k]) )

    cs = CF( 
        (
        (1/2) * (gradg[0,0,0] + gradg[0,0,0] - gradg[0,0,0]),
        (1/2) * (gradg[0,0,1] + gradg[0,1,0] - gradg[0,0,1]),
        (1/2) * (gradg[0,0,2] + gradg[0,2,0] - gradg[0,0,2]),

        (1/2) * (gradg[0,1,0] + gradg[1,0,0] - gradg[0,1,0]),
        (1/2) * (gradg[0,1,1] + gradg[1,1,0] - gradg[0,1,1]),
        (1/2) * (gradg[0,1,2] + gradg[1,2,0] - gradg[0,1,2]),

        (1/2) * (gradg[0,2,0] + gradg[2,0,0] - gradg[0,2,0]),
        (1/2) * (gradg[0,2,1] + gradg[2,1,0] - gradg[0,2,1]),
        (1/2) * (gradg[0,2,2] + gradg[2,2,0] - gradg[0,2,2]),

        (1/2) * (gradg[1,0,0] + gradg[0,0,1] - gradg[1,0,0]),
        (1/2) * (gradg[1,0,1] + gradg[0,1,1] - gradg[1,0,1]),
        (1/2) * (gradg[1,0,2] + gradg[0,2,1] - gradg[1,0,2]),

        (1/2) * (gradg[1,1,0] + gradg[1,0,1] - gradg[1,1,0]),
        (1/2) * (gradg[1,1,1] + gradg[1,1,1] - gradg[1,1,1]),
        (1/2) * (gradg[1,1,2] + gradg[1,2,1] - gradg[1,1,2]),

        (1/2) * (gradg[1,2,0] + gradg[2,0,1] - gradg[1,2,0]),
        (1/2) * (gradg[1,2,1] + gradg[2,1,1] - gradg[1,2,1]),
        (1/2) * (gradg[1,2,2] + gradg[2,2,1] - gradg[1,2,2]),

        (1/2) * (gradg[2,0,0] + gradg[0,0,2] - gradg[2,0,0]),
        (1/2) * (gradg[2,0,1] + gradg[0,1,2] - gradg[2,0,1]),
        (1/2) * (gradg[2,0,2] + gradg[0,2,2] - gradg[2,0,2]),

        (1/2) * (gradg[2,1,0] + gradg[1,0,2] - gradg[2,1,0]),
        (1/2) * (gradg[2,1,1] + gradg[1,1,2] - gradg[2,1,1]),
        (1/2) * (gradg[2,1,2] + gradg[1,2,2] - gradg[2,1,2]),

        (1/2) * (gradg[2,2,0] + gradg[2,0,2] - gradg[2,2,0]),
        (1/2) * (gradg[2,2,1] + gradg[2,1,2] - gradg[2,2,1]),
        (1/2) * (gradg[2,2,2] + gradg[2,2,2] - gradg[2,2,2])
    ), dims=(3,3,3) )    
   
    return cs

def Inv_met(u):
    return Inv(u.Reshape((3,3)) ).Reshape((9,1))

def CS2(gf_g):
    cs1 = CS1(gf_g)
    ginv = Inv_met(gf_g).Reshape((3,3))
    # Christoffel symbols of the second kind
    # cs2[i,j,k] = cs1[i,j,l] * ginv[l,k]
    cs2 = CF(
        (
        cs1[0,0,0] * ginv[0,0] + cs1[0,0,1] * ginv[1,0] + cs1[0,0,2] * ginv[2,0] ,
        cs1[0,1,0] * ginv[0,0] + cs1[0,1,1] * ginv[1,0] + cs1[0,1,2] * ginv[2,0] ,
        cs1[0,2,0] * ginv[0,0] + cs1[0,2,1] * ginv[1,0] + cs1[0,2,2] * ginv[2,0] ,

        cs1[1,0,0] * ginv[0,0] + cs1[1,0,1] * ginv[1,0] + cs1[1,0,2] * ginv[2,0] ,
        cs1[1,1,0] * ginv[0,0] + cs1[1,1,1] * ginv[1,0] + cs1[1,1,2] * ginv[2,0] ,
        cs1[1,2,0] * ginv[0,0] + cs1[1,2,1] * ginv[1,0] + cs1[1,2,2] * ginv[2,0] ,

        cs1[2,0,0] * ginv[0,0] + cs1[2,0,1] * ginv[1,0] + cs1[2,0,2] * ginv[2,0] ,
        cs1[2,1,0] * ginv[0,0] + cs1[2,1,1] * ginv[1,0] + cs1[2,1,2] * ginv[2,0] ,
        cs1[2,2,0] * ginv[0,0] + cs1[2,2,1] * ginv[1,0] + cs1[2,2,2] * ginv[2,0] ,

        cs1[0,0,0] * ginv[0,1] + cs1[0,0,1] * ginv[1,1] + cs1[0,0,2] * ginv[2,1] ,
        cs1[0,1,0] * ginv[0,1] + cs1[0,1,1] * ginv[1,1] + cs1[0,1,2] * ginv[2,1] ,
        cs1[0,2,0] * ginv[0,1] + cs1[0,2,1] * ginv[1,1] + cs1[0,2,2] * ginv[2,1] ,

        cs1[1,0,0] * ginv[0,1] + cs1[1,0,1] * ginv[1,1] + cs1[1,0,2] * ginv[2,1] ,
        cs1[1,1,0] * ginv[0,1] + cs1[1,1,1] * ginv[1,1] + cs1[1,1,2] * ginv[2,1] ,
        cs1[1,2,0] * ginv[0,1] + cs1[1,2,1] * ginv[1,1] + cs1[1,2,2] * ginv[2,1] ,

        cs1[2,0,0] * ginv[0,1] + cs1[2,0,1] * ginv[1,1] + cs1[2,0,2] * ginv[2,1] ,
        cs1[2,1,0] * ginv[0,1] + cs1[2,1,1] * ginv[1,1] + cs1[2,1,2] * ginv[2,1] ,
        cs1[2,2,0] * ginv[0,1] + cs1[2,2,1] * ginv[1,1] + cs1[2,2,2] * ginv[2,1] ,

        cs1[0,0,0] * ginv[0,2] + cs1[0,0,1] * ginv[1,2] + cs1[0,0,2] * ginv[2,2] ,
        cs1[0,1,0] * ginv[0,2] + cs1[0,1,1] * ginv[1,2] + cs1[0,1,2] * ginv[2,2] ,
        cs1[0,2,0] * ginv[0,2] + cs1[0,2,1] * ginv[1,2] + cs1[0,2,2] * ginv[2,2] ,

        cs1[1,0,0] * ginv[0,2] + cs1[1,0,1] * ginv[1,2] + cs1[1,0,2] * ginv[2,2] ,
        cs1[1,1,0] * ginv[0,2] + cs1[1,1,1] * ginv[1,2] + cs1[1,1,2] * ginv[2,2] ,
        cs1[1,2,0] * ginv[0,2] + cs1[1,2,1] * ginv[1,2] + cs1[1,2,2] * ginv[2,2] ,

        cs1[2,0,0] * ginv[0,2] + cs1[2,0,1] * ginv[1,2] + cs1[2,0,2] * ginv[2,2] ,
        cs1[2,1,0] * ginv[0,2] + cs1[2,1,1] * ginv[1,2] + cs1[2,1,2] * ginv[2,2] ,
        cs1[2,2,0] * ginv[0,2] + cs1[2,2,1] * ginv[1,2] + cs1[2,2,2] * ginv[2,2] 

        ),
        dims=(3,3,3)
    )

    return cs2
    



In [94]:

# trial and test functions
g, chi, A, K, G, alpha, beta, B, phi = fes.TrialFunction()
dg , dchi, dA, dK, dG, dalpha, dbeta, dB, dphi = fes.TestFunction()



def TraceFree(gf_M, gf_g):
    return gf_M - 1/3 * InnerProduct(gf_M, gf_g) * gf_g





print( alpha.Operators())

def NonlinearForm(fes):
    cs1 = CS1(gf_g)
    cs2 = CS2(gf_g)
    gfinv = Inv_met(gf_g).Reshape((3,3))
    
    RicciTilde =-1/2 * InnerProduct(gfinv, gf_g.Operator("hesse").)
    Ricci = RicciTilde + Ricciphi
    TF = TraceFree(Ricci, gf_g)

    ### define the RHS
    lf = LinearForm(fes)
    # eq for \tilde \gamma
    lf +=  ( -2*gf_alpha*InnerProduct(gf_A, dg)+ InnerProduct(Lie(gf_g, gf_beta), dg )  )* dx 
    # eq for  \chi
    lf += (2/3* gf_chi * (gf_alpha * gf_K  - InnerProduct(grad(gf_chi),CF((1,1,1)) ))  + InnerProduct(gf_beta, grad(gf_chi)))* dchi *dx  
    # eq for \tilde A
    lf +=( InnerProduct(Lie(gf_A, gf_beta), gf_A) +  gf_alpha * (  gf_K * InnerProduct(gf_A, dA)  - 2 * InnerProduct(gf_A.Reshape((3,3))*Inv_met(gf_g).Reshape((3,3))*gf_A.Reshape((3,3)), dA.Reshape((3,3)) ) )  )* dx
    lf += gf_chi * InnerProduct(TF, dA) * dx
    # eq for  K
    lf += grad(dK)*gf_beta*dx 
    lf +=  -InnerProduct(gf_alpha.Operator("hesse"), Id(3)) * dK *dx  
    lf += gf_alpha * dK * ( InnerProduct(Inv_met(gf_g).Reshape((3,3))*gf_A.Reshape((3,3))*Inv_met(gf_g).Reshape((3,3)), gf_A.Reshape((3,3)))  + 1/3 * gf_K * gf_K  ) *dx
    # eq for \tilde G
    for i in [0,1,2]:
        lf +=  InnerProduct(Inv_met(gf_g) ,(gf_beta.Operator("hesse"))[i,:] )*dG[i]*dx
        lf +=  1/3*InnerProduct((gf_beta.Operator("hesse")).Reshape((3,3,3))[i,i,:] ,Inv_met(gf_g).Reshape((3,3))*dG)*dx
    lf += InnerProduct(grad(gf_G)*gf_beta, dG)*dx
    lf += -InnerProduct(grad(gf_beta)*gf_G, dG)*dx
    lf += 2/3*InnerProduct(Grad(gf_beta), Id(3))*InnerProduct(gf_G, dG)*dx
    lf += - 2  * InnerProduct(gf_A.Reshape((3,3) ) * Grad(gf_alpha), dG) *dx 
    for i in [0,1,2]:
        lf += 2 * gf_alpha * InnerProduct( CS2(gf_g)*dG ,Inv_met(gf_g).Reshape((3,3))*gf_A.Reshape((3,3))*Inv_met(gf_g).Reshape((3,3)) ) *dx
    lf += 12 * gf_alpha * InnerProduct( gf_A.Reshape((3,3))*Inv_met(gf_g).Reshape((3,3))*Grad(gf_phi) , Inv_met(gf_g).Reshape((3,3))*dG)*dx
    lf +=  - gf_alpha * 4/3 *InnerProduct(Inv_met(gf_g).Reshape((3,3)) * Grad(gf_K), dG)*dx
    # eq for \alpha
    lf += -2*gf_alpha*gf_K * dalpha *dx
    # eq for \beta
    lf += InnerProduct(dbeta, gf_B)*dx
    # eq for B
    lf += -InnerProduct(grad(gf_B), grad(dB))*dx

    

    return lf

lf = NonlinearForm(fes)
lf.Assemble()


#blf.Apply(gf_in.vec, gf_out.vec)



['Grad', 'Gradboundary', 'hesse', 'hesseboundary', 'dual']
coef class ngcomp::GridFunctionCoefficientFunction, real, dims = 9 x 3

coef class ngcomp::GridFunctionCoefficientFunction, real, dims = 9 x 3

coef class ngcomp::GridFunctionCoefficientFunction, real, dims = 9 x 3



In [95]:
#A = sp.csr_matrix(blf.mat.CSR())
#plt.rcParams['figure.figsize'] = (10,10)
#plt.spy(A)
#plt.show()

In [96]:

setphi = LinearForm(gf_phi.space)
setphi += -1* ds*log(gf_chi)/4*dx 
setphi.Assemble()


In [97]:
inv_mass = inv_g + inv_chi + inv_A + inv_K + inv_G + inv_alpha + inv_beta + inv_B




In [98]:
dt = 0.01
t = 0
tend = 1
scene1 = Draw(Norm(gf.components[0]), mesh, "g",  clipping= {"z":-1})
#scene2 = Draw(Norm(gf.components[1]), mesh, "chi",  clipping= {"z":-1})
#scene3 = Draw(Norm(gf.components[2]), mesh, "A",  clipping= {"z":-1})
#scene4 = Draw(Norm(gf.components[3]), mesh, "K",  clipping= {"z":-1})
#scene5 = Draw(Norm(gf.components[4]), mesh, "G",  clipping= {"z":-1})
#scene6 = Draw(Norm(gf.components[5]), mesh, "alpha",  clipping= {"z":-1})
#scene7 = Draw(Norm(gf.components[6]), mesh, "beta",  clipping= {"z":-1})
#scene8 = Draw(Norm(gf.components[7]), mesh, "B",  clipping= {"z":-1})

input("start")

with TaskManager():
    while t > tend:
        t += dt
        print("t=",int(t/tend * 100),end="\r")
        lf.Assemble()
        gf.vec.data += dt*inv_mass *lf.vec
        setphi.Assemble()
        gf.components[8].vec.data = mass_phi_inv* setphi.vec
        scene1.Redraw()#Draw(Norm(gf.components[1]), mesh, "chi",  clipping= {"z":-1})
        #scene2.Redraw()
        #scene3.Redraw()
        #scene4.Redraw()
        #scene5.Redraw()
        #scene6.Redraw()
        #scene7.Redraw()
        #scene8.Redraw()


    


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [99]:
Draw(gf.components[0], mesh, "g",  clipping= {"z":-1})
Draw(Norm(gf.components[1]), mesh, "chi",  clipping= {"z":-1})
Draw(Norm(gf.components[2]), mesh, "A",  clipping= {"z":-1})
Draw(Norm(gf.components[3]), mesh, "K",  clipping= {"z":-1})
Draw(Norm(gf.components[4]), mesh, "G",  clipping= {"z":-1})
Draw(Norm(gf.components[5]), mesh, "alpha",  clipping= {"z":-1})
Draw(Norm(gf.components[6]), mesh, "beta",  clipping= {"z":-1})
Draw(Norm(gf.components[7]), mesh, "B",  clipping= {"z":-1})
Draw(Norm(gf.components[8]), mesh, "phi",  clipping= {"z":-1})


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene